In [11]:
#https://medium.com/unit8-machine-learning-publication/computing-the-jacobian-matrix-of-a-neural-network-in-python-4f162e5db180
import numpy as np
N = 500  # Input size
H = 100  # Hidden layer size
M = 20   # Output size
w1 = np.random.randn(N, H)  # first affine layer weights
b1 = np.random.randn(H)     # first affine layer bias
w2 = np.random.randn(H, M)  # second affine layer weights
b2 = np.random.randn(M)     # second affine layer bias

In [12]:
import tensorflow as tf
from keras.layers import Dense
import keras
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
model = keras.models.Sequential()
model.add(Dense(H, activation='relu', kernel_initializer='random_uniform', use_bias=True, input_dim=N))
model.add(Dense(M, activation='softmax', kernel_initializer='random_uniform', use_bias=True, input_dim=M))


Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [13]:
model.summary()
All_parameters = model.get_weights()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               50100     
_________________________________________________________________
dense_4 (Dense)              (None, 20)                2020      
Total params: 52,120
Trainable params: 52,120
Non-trainable params: 0
_________________________________________________________________


In [14]:
def jacobian_tensorflow(x):    
    jacobian_matrix = []
    for m in range(M):
        # We iterate over the M elements of the output vector
        grad_func = tf.gradients(model.output[:, m], model.input)
        gradients = sess.run(grad_func, feed_dict={model.input: x.reshape((1, x.size))})
        jacobian_matrix.append(gradients[0][0,:])
        
    return np.array(jacobian_matrix)

def is_jacobian_correct(jacobian_fn, ffpass_fn):
    """ Check of the Jacobian using numerical differentiation
    """
    x = np.random.random((N,))
    epsilon = 1e-5
    """ Check a few columns at random
    """
    for idx in np.random.choice(N, 5, replace=False):
        x2 = x.copy()
        x2[idx] += epsilon
        num_jacobian = (ffpass_fn(x2) - ffpass_fn(x)) / epsilon
        computed_jacobian = jacobian_fn(x)
        
        if not all(abs(computed_jacobian[:, idx] - num_jacobian) < 1e-3): 
            return False
    return True

def ffpass_tf(x):
    """ The feedforward function of our neural net
    """    
    xr = x.reshape((1, x.size))
    return model.predict(xr)[0]

In [15]:
is_jacobian_correct(jacobian_tensorflow, ffpass_tf)
x0=np.random.rand(1,500)
x0.shape

(1, 500)

In [16]:
import time
tic = time.time()
jacobian_tf = jacobian_tensorflow(x0)
tac = time.time()
print('It took %.3f s. to compute the Jacobian matrix' % (tac-tic))

It took 1.175 s. to compute the Jacobian matrix


In [20]:
jacobian_tf.shape, jacobian_tf.size

((20, 500), 10000)

In [18]:
from numpy.linalg import matrix_rank
matrix_rank(jacobian_tf)

19

In [22]:
#Sparsity of matrix: The sparsity of a matrix can be quantified with a score, which is the number of zero values in the matrix divided by the total number of elements in the matrix.
#https://machinelearningmastery.com/sparse-matrices-for-machine-learning/
sparsity = 1.0 - np.count_nonzero(jacobian_tf) / jacobian_tf.size
print(sparsity)

0.0
